In [ ]:
%matplotlib inline

In [ ]:
import torch
import torch.optim
import torch.profiler
import torch.utils.data
import torchvision.datasets
import torchvision.models
import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim
import torch.profiler

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

In [ ]:
ROOT = '.data'

train_data = datasets.MNIST(root=ROOT,
                            train=True,
                            download=True)

mean = train_data.data.float().mean() / 255
std = train_data.data.float().std() / 255

In [ ]:
train_transforms = T.Compose([
                            T.RandomRotation(5, fill=(0,)),
                            T.RandomCrop(28, padding=2),
                            T.ToTensor(),
                            T.Normalize(mean=[mean], std=[std])
                                      ])

test_transforms = T.Compose([
                           T.ToTensor(),
                           T.Normalize(mean=[mean], std=[std])
                                     ])
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=train_transforms)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=test_transforms)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 101511627.54it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 6629217.63it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 29979805.35it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 1448709.41it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 250)
        self.hidden_fc = nn.Linear(250, 100)
        self.output_fc = nn.Linear(100, output_dim)

    def forward(self, x):

        # x = [batch size, height, width]

        batch_size = x.shape[0]

        x = x.view(batch_size, -1)

        # x = [batch size, height * width]

        h_1 = F.relu(self.input_fc(x))

        # h_1 = [batch size, 250]

        h_2 = F.relu(self.hidden_fc(h_1))

        # h_2 = [batch size, 100]

        y_pred = self.output_fc(h_2)

        # y_pred = [batch size, output dim]

        return y_pred, h_2

In [ ]:
INPUT_DIM = 28 * 28
OUTPUT_DIM = 10

model = MLP(INPUT_DIM, OUTPUT_DIM)

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

Profiling

In [ ]:
model.train()

MLP(
  (input_fc): Linear(in_features=784, out_features=250, bias=True)
  (hidden_fc): Linear(in_features=250, out_features=100, bias=True)
  (output_fc): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
def train(data):
    #print(data)
    inputs, labels = data[0].to(device=device), data[1].to(device=device)
    outputs,_ = model(inputs)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/mlp'),
        record_shapes=True,
        with_stack=True)
prof.start()
for step, batch_data in enumerate(train_loader):
    if step >= (1 + 1 + 3) * 2:
        break
    train(batch_data)
    prof.step()
prof.stop()

In [ ]:
print(prof.key_averages())

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         5.21%       3.654ms        97.47%      68.335ms      22.778ms       0.000us         0.00%       1.162ms     387.333us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        67.03%      46.995ms        86.15%      60.399ms      20.133ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         5.21%       3.654ms        97.47%      68.335ms      22.778ms       0.000us         0.00%       1.162ms     387.333us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        67.03%      46.995ms        86.15%      60.399ms      20.133ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls                                       Input Shapes  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------------------------------------------  
                                          ProfilerStep*         5.21%       3.654ms        97.47%      68.335ms      22.778ms       0.000us         0.00%       1.162ms     387.333us             3                                                 []  
enu

In [ ]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         5.21%       3.654ms        97.47%      68.335ms      22.778ms       0.000us         0.00%       1.162ms     387.333us             3  
                               Optimizer.step#Adam.step         1.75%       1.230ms         3.25%       2.280ms     760.000us       0.000us         0.00%     852.000us     284.000us             3  
    autog

In [ ]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         5.21%       3.654ms        97.47%      68.335ms      22.778ms       0.000us         0.00%       1.162ms     387.333us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        67.03%      46.995ms        86.15%      60.399ms      20.133ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         5.21%       3.654ms        97.47%      68.335ms      22.778ms       0.000us         0.00%       1.162ms     387.333us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        67.03%      46.995ms        86.15%      60.399ms      20.133ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::addmm         0.64%     447.000us         0.95%     667.000us      74.111us     213.000us        13.84%     213.000us      23.667us             9  
                                               aten::mm         0.44%     309.000us         0.58%     408.000us      27.200us     205.000us        13.32%     205.000us      13.667us            15  
---------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv /content/log /content/drive/MyDrive

Profiling with Automatic Mixed Precision

In [ ]:
import torch, time, gc

# Timing utilities
start_time = None

def start_timer():
    global start_time
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()
    start_time = time.time()

def end_timer_and_print(local_msg):
    torch.cuda.synchronize()
    end_time = time.time()
    print("\n" + local_msg)
    print("Total execution time = {:.3f} sec".format(end_time - start_time))
    print("Max memory used by tensors = {} bytes".format(torch.cuda.max_memory_allocated()))

In [ ]:
model.train()

MLP(
  (input_fc): Linear(in_features=784, out_features=250, bias=True)
  (hidden_fc): Linear(in_features=250, out_features=100, bias=True)
  (output_fc): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
use_amp = True
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

def train(data):
  inputs, labels = data[0].to(device=device), data[1].to(device=device)
  with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
    outputs,_ = model(inputs)
    loss = criterion(outputs, labels)
  scaler.scale(loss).backward()
  scaler.step(optimizer)
  scaler.update()
  optimizer.zero_grad() # set_to_non

In [ ]:
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/mlp_amp'),
        record_shapes=True,
        with_stack=True)
prof.start()
start_timer()
for step, batch_data in enumerate(train_loader):

    if step >= (1 + 1 + 3) * 2:
        break
    train(batch_data)
    prof.step()
prof.stop()
end_timer_and_print("Mixed precision:")

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:303: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(



Mixed precision:
Total execution time = 5.466 sec
Max memory used by tensors = 22495232 bytes


In [ ]:
print(prof.key_averages())

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         7.10%       5.221ms        96.52%      70.970ms      23.657ms       0.000us         0.00%       1.403ms     467.667us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        62.95%      46.285ms        80.59%      59.257ms      19.752ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         7.10%       5.221ms        96.52%      70.970ms      23.657ms       0.000us         0.00%       1.403ms     467.667us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        62.95%      46.285ms        80.59%      59.257ms      19.752ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls                                       Input Shapes  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------------------------------------------  
                                          ProfilerStep*         7.10%       5.221ms        96.52%      70.970ms      23.657ms       0.000us         0.00%       1.403ms     467.667us             3                                                 []  
enu

In [ ]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         7.10%       5.221ms        96.52%      70.970ms      23.657ms       0.000us         0.00%       1.403ms     467.667us             3  
                               Optimizer.step#Adam.step         2.13%       1.566ms         3.61%       2.653ms     884.333us       0.000us         0.00%     828.000us     276.000us             3  
         

In [ ]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         7.10%       5.221ms        96.52%      70.970ms      23.657ms       0.000us         0.00%       1.403ms     467.667us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        62.95%      46.285ms        80.59%      59.257ms      19.752ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         7.10%       5.221ms        96.52%      70.970ms      23.657ms       0.000us         0.00%       1.403ms     467.667us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        62.95%      46.285ms        80.59%      59.257ms      19.752ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::copy_         1.93%       1.419ms         2.95%       2.169ms       3.866us     383.000us        20.10%     383.000us       0.683us           561  
                                    aten::_foreach_mul_         0.13%      92.000us         0.21%     158.000us      26.333us     181.000us         9.50%     181.000us      30.167us             6  
---------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv /content/log /content/drive/MyDrive